<a href="https://colab.research.google.com/github/vanryuji/sqlalchemy/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sqlalchemy

In [0]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [0]:
# Connecting 

engine = create_engine('sqlite:///:memory:', echo=True)

In [0]:
# Declare mapping

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)


In [5]:
# Create schema

print('before creating, tables:', engine.table_names())
Base.metadata.create_all(engine)
print('after creating, tables:', engine.table_names())

2019-05-09 16:05:49,814 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-09 16:05:49,816 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 16:05:49,821 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-09 16:05:49,824 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 16:05:49,828 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-09 16:05:49,830 INFO sqlalchemy.engine.base.Engine ()
before creating, tables: []
2019-05-09 16:05:49,834 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-09 16:05:49,836 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 16:05:49,840 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-05-09 16:05:49,841 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 16:05:49,843 INFO sqla

In [6]:
# Creating session

Session = sessionmaker(bind=engine)
session = Session()
print('autocommit:', session.autocommit)

autocommit: False


In [7]:
# Insert

ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)
session.commit()
our_user = session.query(User).filter_by(name='ed').all()
print(our_user)

2019-05-09 16:05:49,887 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-09 16:05:49,896 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-05-09 16:05:49,900 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2019-05-09 16:05:49,904 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-09 16:05:49,911 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-09 16:05:49,916 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-09 16:05:49,919 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [8]:
# Rollback

ed_user = User(name='ed', fullname='Ed Mickey', nickname='edsnickname')
session.add(ed_user)
session.rollback()
users = session.query(User).filter_by(name='ed').all()
print(users)

2019-05-09 16:05:49,937 INFO sqlalchemy.engine.base.Engine ROLLBACK
2019-05-09 16:05:49,940 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-09 16:05:49,943 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-09 16:05:49,944 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [9]:
# Building a Relationship

class Address(Base):
    __tablename__ = 'addresses'

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")
    
Base.metadata.create_all(engine)
print('tables:', engine.table_names())

2019-05-09 16:05:49,981 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-09 16:05:49,983 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 16:05:49,984 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-05-09 16:05:49,986 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 16:05:49,995 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-05-09 16:05:49,997 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 16:05:49,998 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-09 16:05:50,001 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-09 16:05:50,002 INFO sqlalchemy.engine.base.Engine ()
tables: ['addresses', 'users']


In [10]:
# Join

jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')
print('jack addresses:', jack.addresses)

jack.addresses = [Address(email_address='jack@google.com'),
                  Address(email_address='j25@yahoo.com')]
session.add(jack)
session.commit()
user = session.query(User).filter_by(name='jack').one()
print(user.addresses)

jack addresses: []
2019-05-09 16:05:50,035 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-05-09 16:05:50,039 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2019-05-09 16:05:50,044 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-05-09 16:05:50,045 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 2)
2019-05-09 16:05:50,047 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-05-09 16:05:50,049 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 2)
2019-05-09 16:05:50,052 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-09 16:05:50,055 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-09 16:05:50,057 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-09